<a href="https://colab.research.google.com/github/nghess/makemore/blob/main/makemore_lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import urllib3
import random 
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

## Import text file

In [2]:
# Grab text file from github
http = urllib3.PoolManager()
textfile = http.request('GET', 'https://raw.githubusercontent.com/nghess/makemore/master/names.txt')
words = str(textfile.data, 'utf-8').splitlines()

In [3]:
# Take a peak at text file
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [4]:
# Build character to/from string mappings
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [5]:
# Build the dataset
def build_dataset(words):
  block_size = 3  # context length: how many characters do we use to predict the next one?
  X, Y = [], []
  for w in words:

    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
    # print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] #  Crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
len(words)

32033

In [7]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 27), generator=g)
W1 = torch.randn((81, 300), generator=g) * (5/3)/(90**0.5)  # Kaiming init
b1 = torch.randn(300, generator=g) * 0.01
W2 = torch.randn((300, 27), generator=g) * 0.01
b2 = torch.randn(27, generator=g) * 0
parameters = [C, W1, b1, W2, b2]

for p in parameters:
  p.requires_grad = True

In [8]:
sum(p.nelement() for p in parameters)

33456

In [9]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [10]:
lri = []
lossi = []
stepi = []

In [11]:
W1.shape[0]

81

In [ ]:
iters = 100000
ix = 0
# Forward pass
for i in range(iters):

  # Minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (64,), generator=g)

  # Forward pass
  emb = C[Xtr[ix]]  # (32, 2, 2) by indexing into the dataset 
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  h = torch.tanh(hpreact)
  logits = h @ W2 + b2  # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix])

  # Backward pass
  for p in parameters:
      p.grad = None
  loss.backward()

  # Update
  lr = 0.01 
  for p in parameters:
    p.data += -lr * p.grad

  # Track loss
  #lri.append(lr)
  stepi.append(i)
  lossi.append(loss.log10().item())

#print(loss.item())

In [ ]:
emb.shape, W1.shape[0]

In [ ]:
plt.plot(stepi, lossi)

In [ ]:
emb = C[Xtr]  # (32, 2, 2) by indexing into the dataset 
h = torch.tanh(emb.view(-1, W1.shape[0]) @ W1 + b1)
logits = h @ W2 + b2  # (32, 27)
loss = F.cross_entropy(logits, Ytr)
loss

In [ ]:
emb = C[Xdev]  # (32, 2, 2) by indexing into the dataset 
h = torch.tanh(emb.view(-1, W1.shape[0]) @ W1 + b1)
logits = h @ W2 + b2  # (32, 27)
loss = F.cross_entropy(logits, Ydev)
loss

In [ ]:
from numpy.ma import outer
#g = torch.Generator().manual_seed(2147483647 + 10)
block_size = 3 

for _ in range(20):

  out=[]
  context = [0] * block_size  # intialize with all...
  while True:
    emb = C[torch.tensor([context])]
    h = torch.tanh(emb.view(1, -1) @ W1 + b1)
    logits = h @ W2 + b2
    probs = F.softmax(logits, dim=1)
    ix = torch.multinomial(probs, num_samples=1, generator=g).item()
    context = context[1:] + [ix]
    out.append(ix)
    if ix == 0:
      break

  print(''.join(itos[i] for i in out[:-1]))